In [ ]:
# Show plots in Jupyter notebooks
%matplotlib inline

# Reload modules whenever they change
# (for development purposes)
%load_ext autoreload
%autoreload 2

# Make clusterking package available even without installation
import sys
sys.path = ["../../"] + sys.path

In [ ]:
import clusterking as ck
from clusterking.stability.subsamplestability import SubSampleStabilityTester
from clusterking.stability.fom import *
from clusterking.stability.preprocessor import *
from clusterking.cluster import HierarchyCluster
from clusterking.benchmark import Benchmark
import numpy as np
import pandas as pd
import tqdm

In [ ]:
d = ck.Data("output/tutorial_basics.sql")

## SubSampleStabilityTester

In [ ]:
ssst = SubSampleStabilityTester()
ssst.set_fraction(0.999)
ssst.set_repeat(100)
ssst.add_fom(
    MatchingClusters(
        preprocessor=TrivialClusterMatcher(),
        name="MatchingClusters"
    )
)
ssst.add_fom(
    AverageBMProximityFOM(
        preprocessor=TrivialClusterMatcher(),
        name="AverageBMProximity"
    )
)
ssst.add_fom(DeltaNClusters(name="DeltaNClusters"))

In [ ]:
c = HierarchyCluster()
c.set_max_d(0.2)
b = Benchmark()
b.set_metric("euclidean")

In [ ]:
r = ssst.run(d, c, benchmark=b)

In [ ]:
r.df.hist(["MatchingClusters", "DeltaNClusters", "AverageBMProximity"], density=True, layout=(1, 3), figsize=(15, 5));

In [ ]:
r.df.describe()

## SubSampleStabilityVsFraction

In [ ]:
from clusterking.stability.subsamplestability import SubSampleStabilityVsFraction

In [ ]:
ssst = SubSampleStabilityTester()
ssst.set_repeat(repeat=100)
ssst.add_fom(
    MatchingClusters(
        preprocessor=TrivialClusterMatcher(),
        name="MatchingClusters"
    )
)
ssst.add_fom(DeltaNClusters(name="DeltaNClusters"))

In [ ]:
sssvf = SubSampleStabilityVsFraction()

In [ ]:
r = sssvf.run(d, c, ssst, list(np.linspace(0.7, 0.999, 5)))

In [ ]:
r.df.plot.scatter("fraction", "MatchingClusters")

## Noisy sampling

In [ ]:
from clusterking.stability.noisysamplestability import NoisySampleStabilityTester

In [ ]:
import flavio
import numpy as np

s = ck.scan.WilsonScanner(scale=5, eft="WET", basis="flavio")

def dBrdq2(w, q):
    return flavio.np_prediction("dBR/dq2(B+->Dtaunu)", w, q)

s.set_dfunction(
    dBrdq2,
    binning=np.linspace(3.2, 11.6, 10),
    normalize=True,
    variable="q2"  # only sets name of variable
)

s.set_spoints_equidist(
    {
        "CVL_bctaunutau": (-1, 1, 3),
        "CSL_bctaunutau": (-1, 1, 3),
        "CT_bctaunutau": (-1, 1, 3)
    }
)

In [ ]:
d = ck.Data()

In [ ]:
c = HierarchyCluster()
c.set_max_d(0.2)

In [ ]:
nsst = NoisySampleStabilityTester()
nsst.set_repeat(10)
nsst.set_noise("gauss", mean=0., sigma=0.02)
nsst.add_fom(
    MatchingClusters(
        preprocessor=TrivialClusterMatcher(),
        name="MatchingClusters"
    )
)
nsst.add_fom(DeltaNClusters(name="DeltaNClusters"))

In [ ]:
r = nsst.run(d, s, c)

In [ ]:
r.df

In [ ]:
MatchingClusters(
    preprocessor=TrivialClusterMatcher()
).run(r._cached_data[0], r._cached_data[9]).fom

In [ ]:
r._cached_data[0].df.iloc[1][r._cached_data[0].par_cols].tolist()

In [ ]:
r._cached_data[0].par_cols

In [ ]:
r._cached_data[1].plot_clusters_scatter(["CSL_bctaunutau", "CT_bctaunutau", "CVL_bctaunutau"]);

In [ ]:
r._cached_data[0].plot_clusters_scatter(["CSL_bctaunutau", "CT_bctaunutau", "CVL_bctaunutau"]);

In [ ]:
r._cached_data[0].df